In [6]:
import pandas as pd
import mc_processor
import os

In [7]:
## toggle

# survey_year = input("What year is the survey you want to analyze from? (2022, 2023)")
survey_year = str(2022)

In [8]:
# read results files
codebook = pd.read_csv(os.path.join(survey_year,'raw/levels_codebook.csv'), encoding='utf-8')
data = pd.read_csv(os.path.join(survey_year,'raw/data.csv'), encoding='utf-8')
q_codebook = pd.read_csv(os.path.join(survey_year,'raw/question_codebook.csv'), encoding='utf-8', index_col = 'qid')

# q_codebook into dict
q_codebook = q_codebook.to_dict().get('qidFull')

In [9]:
nulldata = pd.DataFrame(data.isnull().sum(),columns=["Sum_Null"]) ##2203 rows in entire dataset
nulldata.sort_values(by=["Sum_Null"],ascending=False,inplace=True)

# nulldata[nulldata["Sum_Null"]>0].head(20)

Use `get_percents` to get percent who selected each option, works for questions that have multiple or single selection.

Demo input optional. For all demo categories, see next cell.

In [10]:
# all demo categories
for key in q_codebook:
    if "dem" in key:
        print(key)

xdemAll
BPCdem1
BPCdem2
BPCxdem1
BPCxdem2
BPCxdem3
BPCxdem4
BPCxdem5
BPCxdem6
BPCxdem7
BPCxdem8
BPCxdem9
demAgeGeneration
demBlackBin
demRaceOther
xBPC7dem
xdemBidenApprove
xdemBidenApprove2
xdemBidenFav
xdemBidenFavFull
xdemEmploy
xdemEvang
xdemGender
xdemHispBin
xdemIdeo3
xdemInc3
xdemMilHH1
xdemReligion
xdemReligOther
xdemUsr
xdemWhite


In [42]:
# testing on range of questions

questions = []
for i in range(6,7): #conjoint
    questions.append("BPC"+str(i))

for question in questions:
    
    #print question text
    for key in q_codebook:
        if key.startswith(question):
            print(question)
            print(q_codebook[key])
            if '---' in q_codebook[key]:
                print(f"\n\n{question}: {q_codebook[key].split('---')[0]}")
                break
            elif '\\\\' in q_codebook[key]:
                q_text = q_codebook[key].split('\\\\')[0]
                print(f"\n\n{question}: {q_text}")
                break


    try:
        display(mc_processor.get_percents(data,codebook,q_codebook,question,demo="xpid3").style.format("{:.0%}"))
    except Exception as e:
        print(f"*Issue with {question} {str(e)[:100]}\n")

BPC6
When you hear about how votes are counted, who of the following would you be more likely to look to for information about the winner of an election?
BPC6
nan


TypeError: argument of type 'float' is not iterable

Figuring out conjoint issue

In [43]:
# get columsn in data that start iwth "BPC6"
data[data.filter(regex='^BPC6').columns]

,BPC6a,BPC6a_cand1n,BPC6a_cand2n,BPC6b,BPC6b_cand1n,BPC6b_cand2n,BPC6c,BPC6c_cand1n,BPC6c_cand2n,BPC6d,BPC6d_cand1n,BPC6d_cand2n,BPC6e,BPC6e_cand1n,BPC6e_cand2n,BPC6f,BPC6f_cand1n,BPC6f_cand2n
0,2,3,10,2,1,6,2,5,7,1,5,4,3,14,3,2,4,5
1,3,14,9,2,13,6,3,2,15,3,3,8,3,4,2,1,5,11
2,3,2,13,3,10,8,3,14,5,3,8,3,3,13,1,3,2,1
3,1,7,15,1,6,3,3,13,1,1,4,14,3,14,11,1,7,3
4,3,12,13,3,5,4,3,9,3,3,15,11,3,4,9,3,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4108,1,13,7,1,6,9,3,5,10,3,9,3,2,6,12,1,11,14
4109,3,8,5,3,14,13,3,2,3,3,12,3,3,1,14,3,15,7
4110,3,8,15,2,1,5,3,15,9,3,4,8,3,6,4,3,5,7
4111,1,3,14,2,11,13,1,14,13,1,9,1,1,14,15,2,8,14


In [37]:
def data_type_check(data,q_columns):
    """
    returns boolean value indicating question type
    """
    matrix = False
    multiple_selections = False

    if len(q_columns) == 0: # underscore only used when each option has selections
        return multiple_selections, matrix # single selection

    ## to address single select questions with a short response text answer
    elif (len(q_columns) == 1) and ("TEXT" in q_columns[0]):
        return multiple_selections, matrix # single selection

    ## selects matrix quetsion (if BOTH multiple columns per question and multiple response categories within each column)
    elif (len(q_columns) > 1) and (len(data[[q_columns[0]]].dropna()[q_columns[0]].unique()) > 2):
        matrix = True
        return multiple_selections, matrix

    ## otherwise, probably mulitple selections
    else:
        multiple_selections = True
        return multiple_selections, matrix

In [39]:
q_columns = data.filter(regex='^'+question+"_").columns

data_type_check(data,q_columns)

data.filter(regex='^' + question + r'[a-z]$').columns

Index(['BPC6a', 'BPC6b', 'BPC6c', 'BPC6d', 'BPC6e', 'BPC6f'], dtype='object')